In [1]:
%matplotlib inline

# Import Packages
import pandas as pd
import numpy  as np
from matplotlib import pyplot as plt

__DESCRIPTION:__ This notebook is used to prepare the Southern Poverty Law Center data for analysis. Cleaning steps proceed as follows. <br>

1. Flag records that indicate militia activity (records marked with an asterisk). <br>
2. Assign each record to the relevant county or counties (some cities are located in more than one county). This is accomplished in two passes. Each pass utilizes a different set database to perform the county assignment. The results from the two passes are merged and flagged for discrepancies. <br>
3. Manual validation of records. Data is exported to Excel. County location(s) and mismatched records are validated through Google search and corrected in the table. Modified table is saved as ...DONE.xlsx and reimported into pandas to verify completeness of county assignments. <br>
4. Assign FIPS codes. <br>
5. Drop records that are listed as occurring STATEWIDE, that were listed as having no location (UNKNOWN) or that indicate a large regions of a state. <br>
6. Save the final data as .csv file. <br>

__INPUTS USED:__ <br>
 - SPLC_Fin.xlsx -- This is the consolidated and reformatted Southern Poverty Law Center data for years 2007-2015, copied from the SPLC Intelligence Report articles. See files Antigov.xlsx and Antigov_Check.xlsx for the files that created this dataset. <br>

 - zip_codes_states.csv -- Used in FIRST PASS of county assignment. Dataset from gaslampmedia.com matching zip code to city and county. (Note, during the cleaning process, errors in the county/city match were observed when an outer join was used to merge this dataset to the SPLC dataset. Since the second pass and manual checks would capture any unmatched records, a dictionary was used to assign county records to the SPLC data instead.) <br>

 - US_SBA_api.csv -- Dataset obtained from the U.S. Small Business Administration API that contains county, city and fips code. This data appears to be of good quality. I split the SPLC data into those whose locations are identified at the county level and those whose locations are at the city level, and merge the city level data with the US_SBA_api data. <br>

 - Checkmap.xlsx -- This is a manual database I created that matched city to county during an earlier pass at cleaning the data. I used this to assign county or counties to records flagged for mismatch or no match. <br>

 - national_county.txt -- This is a database from the U.S. Census Bureau matching county to state and county FIPS codes. <br>
 
__FINAL OUTPUT:__ <br>
 - SPLC_CLEAN.xlsx -- Final SPLC dataset. Used in SPLC to ACS Jupyter Notebook.

In [2]:
# PASS ONE

# Import SPLC data
asplc = pd.read_excel("SPLC_Fin.xlsx")

# Add State Code Abbreviations to the SPLC Dataset
state1 = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
state2 = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',  'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
state_dict = dict(zip(state1, state2))
asplc["ST"] = asplc["State"].map(state_dict)

# Flag Militia locations
asplc["Militia"] = np.where(asplc['Name'].str.contains(r'\*'), "Yes", "No")
asplc["Militia"] = np.where(asplc['Raw'].str.contains(r'\*'), "Yes", asplc["Militia"])
asplc["Militia2"] = np.where(asplc['Name'].str.contains("Militia"), "Yes", asplc["Militia"])

# Remove asterisks from the Raw field where the location is stored
asplc['Raw'] = asplc['Raw'].map(lambda x: x.rstrip(r'\*'))

# Code city_state
asplc['state_city'] = asplc['ST'] + "_" + asplc['Raw']

# Import dataset with City to County conversion
geobase = pd.read_csv("zip_codes_states.csv")
geobase.head()

# Assign state_city code
geobase["County_P1"] = geobase["county"] + " County"
geobase['state_city'] = geobase["state"] + "_" + geobase['city']

# Create dictionary of County to state_city
tryit_dict = dict(zip(geobase.state_city, geobase.County_P1))

# Map county =to city in SPLC dataset using tryit_dict, with state_city as key
asplc['County_P1'] = np.where(~asplc['Raw'].str.contains("County"), asplc['state_city'].map(tryit_dict), asplc['Raw'])

asplc.head()

,State,Name,Year,Raw,ST,Militia,Militia2,state_city,County_P1
0,Alabama,Constitution Party,2007,Cullman,AL,No,No,AL_Cullman,Cullman County
1,Alaska,Constitution Party,2007,Kenai,AK,No,No,AK_Kenai,Kenai Peninsula County
2,Arizona,American Patriot Friends Network,2007,Glendale,AZ,No,No,AZ_Glendale,Maricopa County
3,Arizona,Arizona Citizens Militia,2007,Douglas,AZ,No,Yes,AZ_Douglas,Cochise County
4,Arizona,Arizona Militia,2007,Glendale,AZ,No,Yes,AZ_Glendale,Maricopa County


In [13]:
#PASS TWO:

# GET THE US SBA API DATA READY
# Import API dataset with city, county, and FIPS id
geodf = pd.read_csv("US_SBA_api.csv", index_col = None, usecols =['county_name', 'fips_county_cd', 'full_county_name', 'name', \
                                                              'state_abbreviation', 'state_name'])
# Assign state_city code
geodf['state_city'] = geodf['state_abbreviation'] + "_" + geodf['name']
# Assign state_county code
geodf['state_county'] = geodf['state_abbreviation'] + "_" + geodf['full_county_name']

# Convert FIPS county code into three-digit string
geodf[['fips_county_cd']] = geodf[['fips_county_cd']].astype('str')
geodf['fips_id'] = geodf['fips_county_cd'].str.zfill(3)

# Create dictionary of state_county and fips id
geo_dict = dict(zip(geodf.state_county, geodf.fips_id))



# Import SPLC data
bsplc = pd.read_excel("SPLC_Fin.xlsx")

# Flag Militia locations
bsplc["Militia"] = np.where(bsplc['Name'].str.contains(r'\*'), "Yes", "No")
bsplc["Militia"] = np.where(bsplc['Raw'].str.contains(r'\*'), "Yes", bsplc["Militia"])
bsplc["Militia2"] = np.where(bsplc['Name'].str.contains("Militia"), "Yes", bsplc["Militia"])

# Add State Code Abbreviations to the SPLC Dataset
state1 = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']
state2 = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND',  'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
state_dict = dict(zip(state1, state2))
bsplc["ST"] = bsplc["State"].map(state_dict)

# Remove Asterisks from Raw field
bsplc['Raw'] = bsplc['Raw'].map(lambda x: x.rstrip(r'\*'))
bsplc['state_city'] = bsplc['ST'] + "_" + bsplc['Raw']

# Separate out records that are located in counties versus cities
bcounty = bsplc[bsplc['Raw'].str.contains('County')].copy()
bcity = bsplc[~bsplc['Raw'].str.contains('County')].copy()

# Merge city dataset with US SBA dataset
bcity_fin = pd.merge(bcity, geodf, how = 'inner', left_on = 'state_city', right_on = 'state_city', indicator = 'P2_merge')

# Join the county and city datasets
finbdf = pd.concat([bcounty, bcity_fin], join = 'outer')

# Assign county 
finbdf['County_P2'] = np.where(finbdf.Raw.str.contains('County'), finbdf.Raw, finbdf.full_county_name)

# Get rid of unnecessary columns and make copy of dataframe
finbsplc = finbdf[['State', 'Name', 'Raw', 'Year', 'Militia', 'Militia2', 'County_P2', 'fips_id', 'ST']].copy()
finbsplc.head()

,State,Name,Raw,Year,Militia,Militia2,County_P2,fips_id,ST
26,Georgia,Georgia Militia,Chatham County,2007,No,Yes,Chatham County,NaN,GA
27,Georgia,Georgia Militia,Cobb County,2007,No,Yes,Cobb County,NaN,GA
28,Georgia,Georgia Militia,Gwinnett County,2007,No,Yes,Gwinnett County,NaN,GA
29,Georgia,Georgia Militia,Stephens County,2007,No,Yes,Stephens County,NaN,GA
86,Ohio,Constitutional Militia of Clark County,Clark County,2007,No,Yes,Clark County,NaN,OH


In [14]:
# JOIN PASSES ONE AND TWO

# Create unique label for each record from Name, Year, State and City
asplc['id'] = asplc.Year.map(str) + "_" + asplc.Name + "_" + asplc.ST + "_" + asplc.Raw
finbsplc['id'] = finbsplc.Year.map(str) + "_" + finbsplc.Name + "_" + finbsplc.ST + "_" + finbsplc.Raw

# Merge the Pass 1 and 2 datasets on id
test = pd.merge(asplc, finbsplc.loc[:,('County_P2', 'fips_id', 'id')], how = 'outer', on = 'id', indicator = 'P3_merge')
test.head()

,State,Name,Year,Raw,ST,Militia,Militia2,state_city,County_P1,id,County_P2,fips_id,P3_merge
0,Alabama,Constitution Party,2007,Cullman,AL,No,No,AL_Cullman,Cullman County,2007_Constitution Party_AL_Cullman,Cullman County,043,both
1,Alaska,Constitution Party,2007,Kenai,AK,No,No,AK_Kenai,Kenai Peninsula County,2007_Constitution Party_AK_Kenai,Kenai Peninsula Borough,122,both
2,Arizona,American Patriot Friends Network,2007,Glendale,AZ,No,No,AZ_Glendale,Maricopa County,2007_American Patriot Friends Network_AZ_Glendale,Maricopa County,013,both
3,Arizona,Arizona Citizens Militia,2007,Douglas,AZ,No,Yes,AZ_Douglas,Cochise County,2007_Arizona Citizens Militia_AZ_Douglas,NaN,NaN,left_only
4,Arizona,Arizona Militia,2007,Glendale,AZ,No,Yes,AZ_Glendale,Maricopa County,2007_Arizona Militia_AZ_Glendale,Maricopa County,013,both


In [16]:
# CHECK FOR DISCREPANCIES IN THE MATCHES AND ASSIGN THE COUNTY

# Create a column to flag mismatches between the two passes
test['flag_p12'] = np.where(test.County_P1 == test.County_P2, 'Match', \
                        np.where((pd.isnull(test.County_P1)) & (pd.isnull(test.County_P2)), "No Match", "Mismatch"))
#test1[(test1._merge != 'both') & (test1.flag == 'No Match')]
#test.groupby('flag_p12').count()

# Assign the County based on the flag
test['County_P3'] = np.where(test.flag_p12 == 'Match', test.County_P2, \
                               np.where((~pd.isnull(test.County_P2)) & (pd.isnull(test.County_P1)), test.County_P2, \
                                                np.where((~pd.isnull(test.County_P1)) & (pd.isnull(test.County_P2)), test.County_P1, \
                                                        np.where((test.flag_p12 == 'Mismatch'), "Error", "No Match"))))

# Export to excel for manual validation
test.to_excel('SPLC_CheckIt.xlsx', encoding = 'utf-8')

In [21]:
# IMPORT CHECK ONE RESULTS
df= pd.read_excel('SPLC_CheckIt_Done.xlsx')

# Drop records flagged for drop (due to no match found or erroneous record)
check1 = df[df['Check'] != 'dropit'].copy()

# Determine county assignment based on flags
check1['County_P4'] = np.where(check1['Check'] == 'Keep2', check1.County_P2, \
                               np.where(check1['Check'] == 'st_unk', 'None', check1.County_P3))
# check1[check1['Check'] == 'Keep2']

# Import checkmap table
checkmap = pd.read_excel('Check_map.xlsx', usecols = ['state_city', 'Final', 'Fix2', 'Note'])

# Flag only those records that are useful
checkmap1 = checkmap[(checkmap['Final'] == 1) | (checkmap['Final'] == 3)]
# Drop duplicate records that will cause unnecessary overmatch
checkmap1 = checkmap1.drop_duplicates()

# Merge the checkmap records to the check1 dataset to resolve whatever discrepancies can be resolved
testcheck = pd.merge(check1, checkmap1, how = 'left', on = 'state_city', indicator = 'testcheck_merge')
# Assign county based on the check results
testcheck['County_P5'] = np.where((testcheck.County_P4 == 'No Match') & (~pd.isnull(testcheck.Fix2)), testcheck.Fix2, \
                                 np.where(((testcheck.County_P4 != 'No Match') | (testcheck.County_P4 != 'None')), testcheck.County_P4, \
                                          np.where((testcheck.County_P4 == 'None'), "None", "Error")))

# Export to Excel for Manual Check #2
testcheck.to_excel('SPLC_CheckIt_Twice.xlsx')

In [22]:
# Import the results from the Manual Check #2
check2 = pd.read_excel('SPLC_CheckIt_Twice_Done.xlsx')

# Assign county based on the results from Check #2
check2['County_P6'] = np.where((check2.County_P5 == 'No Match') & (check2.Check == 'Keep2'), check2.County_P2, check2.County_P5)
 
#check2[(check2['County_P6'] != 'None') & (check2['County_P6'] != 'No Match')]

,Column2,Column1,State,Name,Year,Raw,ST,Militia,Militia2,state_city,...,P3_merge,flag_p12,County_P3,County_P4,Final,Fix2,Note,testcheck_merge,County_P5,County_P6
0,0,16,Colorado,American Freedom Network,2007,Johnstown,CO,No,No,CO_Johnstown,...,both,Match,Weld County,Weld County,NaN,NaN,NaN,left_only,Weld County,Weld County
1,1,8,California,American Independent Party,2007,Lemon Grove,CA,No,No,CA_Lemon Grove,...,both,Match,San Diego County,San Diego County,NaN,NaN,NaN,left_only,San Diego County,San Diego County
2,2,135,Wisconsin,American Opinion Book Services,2007,Appleton,WI,No,No,WI_Appleton,...,both,Match,Outagamie County,Outagamie County,NaN,NaN,NaN,left_only,Outagamie County,Outagamie County
3,3,136,Wisconsin,American Opinion Book Services,2007,Appleton,WI,No,No,WI_Appleton,...,both,Mismatch,Error,Calumet County,NaN,NaN,NaN,left_only,Calumet County,Calumet County
4,4,137,Wisconsin,American Opinion Book Services,2007,Appleton,WI,No,No,WI_Appleton,...,both,Mismatch,Error,Winnebago County,NaN,NaN,NaN,left_only,Winnebago County,Winnebago County
5,5,2,Arizona,American Patriot Friends Network,2007,Glendale,AZ,No,No,AZ_Glendale,...,both,Match,Maricopa County,Maricopa County,NaN,NaN,NaN,left_only,Maricopa County,Maricopa County
6,6,3,Arizona,Arizona Citizens Militia,2007,Douglas,AZ,No,Yes,AZ_Douglas,...,left_only,Mismatch,Cochise County,Cochise County,NaN,NaN,NaN,left_only,Cochise County,Cochise County
7,7,4,Arizona,Arizona Militia,2007,Glendale,AZ,No,Yes,AZ_Glendale,...,both,Match,Maricopa County,Maricopa County,NaN,NaN,NaN,left_only,Maricopa County,Maricopa County
9,6688,6233,Minnesota,We Are Change,2014,Cedar Falls,MN,No,No,MN_Cedar Falls,...,left_only,No Match,No Match,No Match,NaN,NaN,NaN,left_only,No Match,Black Hawk County
10,10,121,Texas,Border Rescue,2007,Richardson,TX,No,No,TX_Richardson,...,both,Match,Dallas County,Dallas County,NaN,NaN,NaN,left_only,Dallas County,Dallas County


In [23]:
# NOW ASSIGN FIPS codes

# Import dataset with County to FIPS code conversion
county_FIPS = pd.read_csv("national_county.txt", header = None, names = ["ST", "ID1", "ID2", "County", "ID3"], dtype = object)
county_FIPS["state_county"] = county_FIPS["ST"] + "_" + county_FIPS["County"]
county_FIPS['fips'] = county_FIPS.ID1.map(str) + county_FIPS.ID2.map(str)
county_FIPS.head()

# Create dictionary of County to FIPS code
tryit2_dict = dict(zip(county_FIPS.state_county, county_FIPS.fips))

# Create state county field
check2['state_county'] = check2.ST + "_" + check2.County_P6

# Map state_county to to SPLC dataset using tryit2_dict, with state_county as key
check2["fips1"] = check2["state_county"].map(tryit2_dict)

# Export to Excel for final check for those records that should be dropped (ex. locations listed as Statewide or Unknown 
# or are unable to be matched.)
check2.to_excel("SPLC_CheckIt_Thrice.xlsx")

In [25]:
# Import results from final check
df = pd.read_excel('SPLC_CheckIt_Thrice_Done.xlsx')

# Drop records that were verified for removal
check3 = df[df['Check'] != 'dropit'].copy()

# Assign final county based on results from the final check
check3['County_P7'] = np.where((~pd.isnull(check3.Add)), check3.Add, check3.County_P6)

# Create state county field
check3['state_county3'] = check3.ST + "_" + check3.County_P7

# Map fips to newly assigned county records using tryit2_dict, with state_county as key
check3["fips2"] = check3["state_county3"].map(tryit2_dict)

#check3[(pd.isnull(check3.fips2)) & ((check3.County_P7 != 'None') & (check3.County_P7 !='No Match'))]

# Filter those records that have a fips assigned (no fips means it is a deliberate no match)
done = check3[~pd.isnull(check3.fips2)][['State', 'Name', 'Year', 'Raw', 'ST',\
                                         'Militia', 'Militia2', 'County_P7', 'fips2' ]]

# Export results to Excel for final analysis
done.to_excel('SPLC_CLEAN.xlsx')

,Column3,Column2,Column1,State,Name,Year,Raw,ST,Militia,Militia2,...,County_P5,County_P6,state_county,fips1,fips_id,Check,Add,County_P7,state_county3,fips2
0,5961,5961,5806,Alaska,Bethel Citizens Militia*,2014,Tuluksak,AK,Yes,Yes,...,Bethel County,Bethel County,AK_Bethel County,NaN,NaN,Keep2,Bethel Census Area,Bethel Census Area,AK_Bethel Census Area,02050
1,7976,7976,7852,Alaska,American Patriots III%,2016,Statewide,AK,No,No,...,None,None,AK_None,NaN,NaN,st_unk,NaN,None,AK_None,NaN
2,8030,8030,7853,Alaska,Anchorage Municipality Defense Force*,2016,Anchorage,AK,Yes,Yes,...,Anchorage Municipality,Anchorage Municipality,AK_Anchorage Municipality,2020.0,20.0,Keep2,NaN,Anchorage Municipality,AK_Anchorage Municipality,02020
3,8049,8049,7854,Alaska,Central Alaska Militia*,2016,Delta Junction,AK,Yes,Yes,...,Southeast Fairbanks Census Area,Southeast Fairbanks Census Area,AK_Southeast Fairbanks Census Area,2240.0,240.0,Keep2,NaN,Southeast Fairbanks Census Area,AK_Southeast Fairbanks Census Area,02240
4,8066,8066,7855,Alaska,Constitution Party of Alaska,2016,Soldotna,AK,No,No,...,Kenai Peninsula Borough,Kenai Peninsula Borough,AK_Kenai Peninsula Borough,2122.0,122.0,Keep2,NaN,Kenai Peninsula Borough,AK_Kenai Peninsula Borough,02122
5,8112,8112,7856,Alaska,Eagle Forum,2016,Cordova,AK,No,No,...,Valdez-Cordova Census Area,Valdez-Cordova Census Area,AK_Valdez-Cordova Census Area,2261.0,261.0,Keep2,NaN,Valdez-Cordova Census Area,AK_Valdez-Cordova Census Area,02261
6,8183,8183,7850,Alaska,III% United Patriots*,2016,Statewide,AK,Yes,Yes,...,None,None,AK_None,NaN,NaN,st_unk,NaN,None,AK_None,NaN
7,8341,8341,7857,Alaska,Oath Keepers,2016,Anchorage,AK,No,No,...,Anchorage Municipality,Anchorage Municipality,AK_Anchorage Municipality,2020.0,20.0,Keep2,NaN,Anchorage Municipality,AK_Anchorage Municipality,02020
8,8535,8535,7858,Alaska,South Central Patriots*,2016,Wasilla,AK,Yes,Yes,...,Matanuska-Susitna Borough,Matanuska-Susitna Borough,AK_Matanuska-Susitna Borough,2170.0,170.0,Keep2,NaN,Matanuska-Susitna Borough,AK_Matanuska-Susitna Borough,02170
9,8583,8583,7859,Alaska,"Three Percenters-III%ers, The",2016,Statewide,AK,No,No,...,None,None,AK_None,NaN,NaN,st_unk,NaN,None,AK_None,NaN
